In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

In [2]:
# util functions:

# Define SPARQL endpoint
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

def query_authors(disease_entity):
    query = f'''
    PREFIX target: <http://www.wikidata.org/entity/{disease_entity}>

    SELECT
      ?count
      ?author ?authorLabel ?authorDescription (CONCAT("/author/", SUBSTR(STR(?author), 32)) AS ?authorUrl)
      (COALESCE(?orcid_, CONCAT("orcid-search/quick-search/?searchQuery=", ?authorLabel)) AS ?orcid)
    WITH {{
      SELECT
        ?author
        (count(?work) as ?count)
      WHERE {{
        {{ ?work wdt:P921/wdt:P31*/wdt:P279* target: . }}
        UNION
        {{ ?work wdt:P921/wdt:P361+ target: . }}
        UNION
        {{ ?work wdt:P921/wdt:P1269+ target: . }}
        ?work wdt:P50 ?author .
      }}
      GROUP BY ?author
      ORDER BY DESC(?count)
      LIMIT 200
    }} AS %result
    WHERE {{
      INCLUDE %result

      # Include optional ORCID iD
      OPTIONAL {{ ?author wdt:P496 ?orcid_ . }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh" . }}
    }}
    ORDER BY DESC(?count)
    '''

    # Set the query and format to JSON
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the query and convert the results to a Pandas DataFrame
    results = sparql.query().convert()
    df = pd.json_normalize(results["results"]["bindings"])
    df["count.value"] = pd.to_numeric(df["count.value"])

    # Create the bar chart using Plotly
    fig = px.bar(df, x='authorLabel.value', y='count.value').update_layout(
    title='Publication Count by Author',
    yaxis=dict(title='Count'),
    xaxis=dict(title='Author'))

    return fig

# query_authors('Q131755')

In [3]:
def publications_per_year(disease):
    # Define SPARQL endpoint
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Define SPARQL query
    query = f"""
    PREFIX target: <http://www.wikidata.org/entity/{disease}>

    # Inspired from LEGOLAS - http://abel.lis.illinois.edu/legolas/
    # Shubhanshu Mishra, Vetle Torvik
    select ?year (count(?work) as ?number_of_publications) where {{
      {{
        select (str(?year_) as ?year) (0 as ?pages) where {{
          # default values = 0
          ?year_item wdt:P31 wd:Q577 .
          ?year_item wdt:P585 ?date .
          bind(year(?date) as ?year_)
          {{
            select (min(?year_) as ?earliest_year) where {{
              {{ ?work wdt:P921/wdt:P31*/wdt:P279* target: . }}
              union {{ ?work wdt:P921/wdt:P361+ target: . }}
              union {{ ?work wdt:P921/wdt:P1269+ target: . }}
              ?work wdt:P577 ?publication_date .
              bind(year(?publication_date) as ?year_)
            }}
          }}
          bind(year(now()) as ?next_year)
          filter (?year_ >= ?earliest_year && ?year_ <= ?next_year)
        }}
      }}
      union {{
        select ?work (min(?years) as ?year) where {{
          {{ ?work wdt:P921/wdt:P31*/wdt:P279* target: . }}
          union {{ ?work wdt:P921/wdt:P361+ target: . }}
          union {{ ?work wdt:P921/wdt:P1269+ target: . }}
          ?work wdt:P577 ?dates .
          bind(str(year(?dates)) as ?years) .
        }}
        group by ?work
      }}
    }}
    group by ?year
    order by ?year
    """

    # Set the query and format to JSON
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the query and convert the results to a Pandas DataFrame
    results = sparql.query().convert()
    df = pd.json_normalize(results["results"]["bindings"])
    df["year.value"] = pd.to_numeric(df["year.value"])
    df["number_of_publications.value"] = pd.to_numeric(df["number_of_publications.value"])

    # Create the bar chart using Plotly
    fig = px.bar(df, x="year.value", y="number_of_publications.value").update_layout(
    title='Publications per year',
    yaxis=dict(title='Count'),
    xaxis=dict(title='Year'))

    return fig

# publications_per_year('Q131755')

In [4]:
# Create a list of disease options for the dropdown menu
disease_options = [
    {'label': 'COVID-19', 'value': 'Q84263196'},
    {'label': 'Schizophrenia', 'value': 'Q41112'},
    {'label': 'Bipolar Disorder', 'value': 'Q131755'}
]

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Disease Dashboard'),
    dcc.Dropdown(
        id='disease-dropdown',
        options=disease_options,
        #value=disease_options[0]['value'],
        placeholder='Select a disease'
    ),
    html.Div(id='page-content')
])

# Step 4: Define callbacks
@app.callback(Output('page-content', 'children'),
              Input('disease-dropdown', 'value'))

def update_page_content(selected_disease):
    if selected_disease == 'Q84263196':
        return generate_covid_content()
    elif selected_disease == 'Q41112':
        return generate_schizophrenia_content()
    elif selected_disease == 'Q131755':
        return generate_bip_dis_content()
    # Add more conditions for other diseases

    # If no disease is selected or unknown disease, display a default message
    else:
        return html.Div('Please select a disease')

# Define functions to generate content for each disease
def generate_covid_content():

    publications_per_year_covid = publications_per_year('Q84263196')

    content = [
            html.H2("Covid"),
            query_authors('Q84263196'),
            dcc.Graph(id="publications_per_year_covid", figure=publications_per_year_covid)
        ]
    return html.Div(content)


def generate_schizophrenia_content():

    publications_per_year_schizo = publications_per_year('Q41112')

    content = [
            html.H2("Schizophrenia"),
            query_authors('Q41112'),
            dcc.Graph(id="publications_per_year_schizo", figure=publications_per_year_schizo)
        ]
    return html.Div(content)

def generate_bip_dis_content():
    # Code to generate the content for disease 1
    # This can include calling functions to fetch data, creating plots, etc.
    # Return the generated content as a Dash component or a list of components

    publications_per_year_bip_dis = publications_per_year('Q131755')
    authors_count_bip_dis = query_authors('Q131755')

    content = [
            html.H2("Bipolar Disorder"),
            #html.P("This is the content for disease 2."),
            query_authors('Q131755'),
            dcc.Graph(id="publications_per_year_bip_dis", figure=publications_per_year_bip_dis),
            dcc.Graph(id="authors_count_bip_dis", figure=authors_count_bip_dis)
        ]
    return html.Div(content)

# Add more functions for other diseases

# Step 6: Run the app
if __name__ == '__main__':
    app.run_server(debug=False)



Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [16/May/2023 14:31:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:31:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:31:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:31:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2023 14:31:50] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\lucys\anaconda3\envs\scholia_visualization\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\lucys\anaconda3\envs\scholia_visualization\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\lucys\anaconda3\envs\scholia_visualization\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\lucys\anaconda3\envs\scholia_visualization\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\lucys\anaconda3\envs\scholia_visualization\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  File "C:\Users\lucys\anaconda3\envs\scholia_visualization\lib\site-packages\dash\_callback.py", lin

127.0.0.1 - - [16/May/2023 14:32:11] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [16/May/2023 14:32:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:29] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2023 14:32:29] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2023 14:32:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:53] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 14:32:53] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2023 14:32:53] "POST /_dash-upda